# Twitchly Quick Start Guide


## Set Up
To start off with understanding Twitchly, we will first dive into the [Twith API](https://dev.twitch.tv/docs/api/).

We will learn about using your `clientID` to make calls to the Twitch API!

First, follow the quick set up guide on the link above to register your application.

For the sake of this demo
we can use the following values when registering:
- Name = APIDemo
- OAuth Redirect URL = http://localhost:5000/
- Category = Analytics Tool

Once you have created your application, you can click `manage` to find your `clientID` and `clientSecret`.

Next, we will use a wrapper python library called [python-twitch-client](https://python-twitch-client.readthedocs.io/en/latest/index.html).
That will tell you how to install the library as well as provide some very useful documentation.
For now, we will run through an example where we use the popular streamer [Ninja](https://en.wikipedia.org/wiki/Ninja_(streamer)
and get his user data to find out who he is following and who those followed channels are following.

In [65]:
from twitch import TwitchClient 
import pandas as pd


# Let's create the client by passing in your specific client id.
client = TwitchClient(client_id = "zjwe67emf2ri3ecyqvihgzhb1r3l4i")

# Most of the API is reliant on user-ids so this useful query can provide us the list of user-ids
# for the given usernames that we are all familiar with.
queried_users = client.users.translate_usernames_to_ids(['Ninja'])

# Now that we have the list of users, let's extract Ninja from the list.
user = queried_users[0]

# The user data is full of great information but for now, let's just verify that we have the username and id.
print(user.display_name, user.id)

Ninja 19571641


In [66]:
# Let's now query for the channels that ninja follows.
# For this example, let's just grab 5 channels.
# Check out the documentation to find out some other functionalities of this function.
followed_channels = client.users.get_follows(ninja.id, limit = 5)

# We filter out the result a little and add them into a list.
channels = []
for channel in followed_channels:
    channels.append(channel.channel)

# For the fun of it, let's throw these results into a Pandas Dataframe.
pd.DataFrame.from_records(channels)

,broadcaster_language,broadcaster_type,created_at,description,display_name,followers,game,id,language,logo,...,partner,privacy_options_enabled,private_video,profile_banner,profile_banner_background_color,status,updated_at,url,video_banner,views
0,en,partner,2016-07-04 00:22:25,Baseballer and Gamer and occasional DJ. @Trevm...,iamtrevormay,46866,Fortnite,128479231,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,Is it fixed!? !subgoal @IamTrevormay on Twitte...,2018-09-22 02:07:13,https://www.twitch.tv/iamtrevormay,https://static-cdn.jtvnw.net/jtv_user_pictures...,417053
1,en,partner,2013-07-21 04:09:19,Check out my YouTube channel: http://www.youtu...,WILDCAT,642483,Fortnite,46386566,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,True,False,False,None,#ffffff,SOLO SCRIMS BC I'M DOODOO AND NEED TO PRACTICE...,2018-09-22 04:28:48,https://www.twitch.tv/wildcat,https://static-cdn.jtvnw.net/jtv_user_pictures...,2824626
2,en,partner,2018-04-13 22:09:57,The home for Beyond The Summit's unique and au...,btssmash,48988,Super Smash Bros. Melee,214062798,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,RERUN: Mang0 vs Hungrybox - Singles LB - Smash...,2018-09-22 04:47:15,https://www.twitch.tv/btssmash,https://static-cdn.jtvnw.net/jtv_user_pictures...,2106262
3,en,affiliate,2009-07-01 13:06:10,Check out my stream for retro style games like...,Zexxios,914,Dead Cells,7054369,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,False,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,#002cbd,"More farming, more 1 cell attempts!",2018-09-22 04:04:10,https://www.twitch.tv/zexxios,https://static-cdn.jtvnw.net/jtv_user_pictures...,9283
4,en,partner,2012-05-11 22:56:04,"Hi, I'm Marcel, but you can call me BasicallyI...",BasicallyIDoWrk,253714,Fortnite,30446023,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,I'm really bad.,2018-09-22 04:30:36,https://www.twitch.tv/basicallyidowrk,https://static-cdn.jtvnw.net/jtv_user_pictures...,988747


In [67]:
# Let's write a quick function that will take in the list of channels,
# query up to k followers for each channel and then add them all into
# a list as output.
def multi_follow_query(channels, k):
    result = []
    for channel in channels:
        followed_channels = client.users.get_follows(channel.id, limit = k)
        for followed_channel in followed_channels:
            result.append(followed_channel.channel)
    return result

# Let's query for results from the list of channels that Ninja is following.
result = multi_follow_query(channels, 2)

pd.DataFrame.from_records(result)

,broadcaster_language,broadcaster_type,created_at,description,display_name,followers,game,id,language,logo,...,partner,privacy_options_enabled,private_video,profile_banner,profile_banner_background_color,status,updated_at,url,video_banner,views
0,en,partner,2018-05-18 18:47:42,Jordan Fisher,JordanFisher,14977,Fortnite,224145872,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,"Squads with Fear, Justin Ola and Train || Last...",2018-09-22 04:01:25,https://www.twitch.tv/jordanfisher,https://static-cdn.jtvnw.net/jtv_user_pictures...,100666
1,en,partner,2015-03-24 22:02:30,I stream games and stuff,GubbaTV,101216,Fortnite,86198803,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,Not heading to Dallas until we hit 1800 subs! ...,2018-09-22 04:04:49,https://www.twitch.tv/gubbatv,https://static-cdn.jtvnw.net/jtv_user_pictures...,5134681
2,en,partner,2016-03-29 16:07:17,"19 year old, fluffy entertainer looking to giv...",Aydan,133946,Fortnite,120244187,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,Ghost Aydan | !SUBTEMBER IS LIVE! l 3 Time Sum...,2018-09-22 04:31:57,https://www.twitch.tv/aydan,https://static-cdn.jtvnw.net/jtv_user_pictures...,519708
3,en,partner,2014-01-04 18:43:58,"Swing by for the mediocre game play, stay for ...",Curvyllama,111615,Fortnite,54411072,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,#052525,I got popcorn and a DREAM,2018-09-22 04:29:32,https://www.twitch.tv/curvyllama,https://static-cdn.jtvnw.net/jtv_user_pictures...,2890915
4,en,partner,2017-11-23 18:21:05,,TwitchPresents2,94119,Always On,182953678,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,True,False,False,None,None,POKÉMON RERUN FRIDAYS AND SATURDAYS at 10 AM P...,2018-09-22 04:29:43,https://www.twitch.tv/twitchpresents2,https://static-cdn.jtvnw.net/jtv_user_pictures...,8649538
5,en,affiliate,2018-03-06 01:49:04,Variety streamer. I play retro games Mon/Wed a...,HarmonyGinger,291,Subnautica,202998830,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,False,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,Spookier than Amnesia,2018-09-22 04:01:37,https://www.twitch.tv/harmonyginger,https://static-cdn.jtvnw.net/jtv_user_pictures...,6951
6,en,partner,2013-07-30 22:49:04,,Thiefs,112197,Fortnite,46865623,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,FaZe Thiefs - Fall Skirmish w/ Wildcat!,2018-09-22 04:31:11,https://www.twitch.tv/thiefs,https://static-cdn.jtvnw.net/jtv_user_pictures...,1173888
7,en,partner,2015-02-03 07:50:36,Battle Royale Pro Player.,cloakzy,593548,Fortnite,81687332,en,https://static-cdn.jtvnw.net/jtv_user_pictures...,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,4Head l00tlake,2018-09-22 04:30:03,https://www.twitch.tv/cloakzy,https://static-cdn.jtvnw.net/jtv_user_pictures...,5441524
